In [ ]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import tensorflow as tf
import keras
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy import stats

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.optimizers import SGD, Adam

import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
train = pd.read_csv("/content/drive/My Drive/data/open_data/train.csv", index_col=0)
train['voted'] = train['voted'] 
test=pd.read_csv('/content/drive/My Drive/data/open_data/test_x.csv', index_col=0)
submission=pd.read_csv('/content/drive/My Drive/data/open_data/sample_submission.csv', index_col=0)  

#voted = 0 , unvoted = 1 으로 바꿔주기
train.voted = train.voted - 1

In [ ]:
Answers_time = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE', 
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE', 
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE', 'voted']

In [ ]:
Answers_time_only = ['QaE', 'QbE', 'QcE', 'QdE', 'QeE',
             'QfE', 'QgE', 'QhE', 'QiE', 'QjE', 
             'QkE', 'QlE', 'QmE', 'QnE', 'QoE', 
             'QpE', 'QqE', 'QrE', 'QsE', 'QtE']

In [ ]:
Answers = ['QaA', 'QbA', 'QcA', 'QdA', 'QeA',
             'QfA', 'QgA', 'QhA', 'QiA', 'QjA', 
             'QkA', 'QlA', 'QmA', 'QnA', 'QoA', 
             'QpA', 'QqA', 'QrA', 'QsA', 'QtA']


#data reverse
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    train[flip] = 6 - train[flip]
#secret data reverse
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    train[flip] = 6 - train[flip]

train['Mach_score'] = train[Answers].mean(axis = 1)

In [ ]:
#data reverse
flipping_columns = ["QeA", "QfA", "QkA", "QqA", "QrA"]
for flip in flipping_columns: 
    test[flip] = 6 - test[flip]
#secret data reverse
flipping_secret_columns = ["QaA", "QdA", "QgA", "QiA", "QnA"]
for flip in flipping_secret_columns: 
    test[flip] = 6 - test[flip]

test['Mach_score'] = test[Answers].mean(axis = 1)

In [ ]:
for i in Answers_time:
  sns.kdeplot(train[i][train.voted == 0], color = 'b', label = "voted")
  sns.kdeplot(train[i][train.voted == 1], color = 'r', label = "unvoted")
  plt.title(i + ' distribution with voted')
  plt.show()

In [ ]:
log_Answer_time = train[Answers_time_only].copy()
log_Answer_time[Answers_time_only] = np.log1p(train[Answers_time_only])
train[Answers_time_only] = log_Answer_time

In [ ]:
log_Answer_time_t = test[Answers_time_only].copy()
log_Answer_time_t[Answers_time_only] = np.log1p(test[Answers_time_only])
test[Answers_time_only] = log_Answer_time_t

In [ ]:
#after log
for i in Answers_time:
  sns.kdeplot(train[i][train.voted == 0], color = 'b', label = "voted")
  sns.kdeplot(train[i][train.voted == 1], color = 'r', label = "unvoted")
  plt.title(i + ' distribution with voted')
  plt.show()


In [ ]:
#missing 확인
print('train : ',train.isnull().sum())
print('test : ',test.isnull().sum())

In [ ]:
label = LabelEncoder()
train['gender_code'] = label.fit_transform(train['gender'])
train['age_group_code'] = label.fit_transform(train['age_group'])
train = pd.concat([train, pd.get_dummies(train['race'], prefix= 'race')], axis = 1, join = 'inner')
train = pd.concat([train, pd.get_dummies(train['religion'], prefix= 'religion')], axis = 1, join = 'inner')

test['gender_code'] = label.fit_transform(test['gender'])
test['age_group_code'] = label.fit_transform(test['age_group'])
test = pd.concat([test, pd.get_dummies(test['race'], prefix= 'race')], axis = 1, join = 'inner')
test = pd.concat([test, pd.get_dummies(test['religion'], prefix= 'religion')], axis = 1, join = 'inner')


In [ ]:
train = train.drop(train[train.familysize >= 20].index)


In [ ]:
plt.pie((train.groupby(train['voted']).size()),
        labels = ['uvoted', 'voted'],
        shadow = True,
        colors = ['r', 'royalblue'],
        autopct='%0.2f%%',
        startangle=90
        )
plt.title("ratio of vote")
plt.axis('equal')
plt.show()

In [ ]:
cat_features = ['age_group', 'education', 'engnat', 'familysize', 'gender', 'hand',
       'married', 'race', 'religion', 'urban']

for i in cat_features:
  pd.crosstab(train[i],train['voted']).plot.bar()
  plt.title(i + ' distribution with voted')

In [ ]:
sns.distplot(train.Mach_score[train.voted == 1], color = 'red', label = "unvoted")
sns.distplot(train.Mach_score[train.voted == 0], color = 'b', label = "voted")

plt.legend(title = 'voted')
plt.show()

In [ ]:
train = train.drop(['age_group', 'gender', 'race', 'religion'], axis = 1)
test = test.drop(['age_group', 'gender', 'race', 'religion'], axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
Scaler = StandardScaler()

train[Answers_time_only] = Scaler.fit_transform(train[Answers_time_only])
test[Answers_time_only] = Scaler.fit_transform(test[Answers_time_only])

In [ ]:
ros = RandomOverSampler()

X = train.drop('voted', axis = 1).values
y = train.voted.values



x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state= 0)
x_train, y_train= ros.fit_resample(x_train, y_train)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(test.values)

print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape )

In [ ]:
params = {
        'objective':'binary:logistic',
        'max_depth':5,
        'learning_rate':0.1, 
        'eval_metric':'auc',
        'min_child_weight':8,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'seed':29,
#         'reg_lambda':1,
        'reg_alpha':0,
        'gamma':1,
        'scale_pos_weight':1,
        'n_estimators': 100,
        'nthread':-1
}

watchlist = [(d_train, 'train'), (d_valid, 'valid')]
nrounds=10000  
model_xgb = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=50, 
                           maximize=True, verbose_eval=10)

In [ ]:
pred_y1 = model_xgb.predict(d_test) + 1


In [ ]:
submission['voted'] = pred_y1

In [ ]:
submission.to_csv('/content/sample_data/sample_submission.csv') 
